In [1]:
import numpy as np
import ast
import utm

In [2]:
def from_latlon(latitude, longitude, geozone_num, geozone_let):
    """
    convert lat, lon to (x, y) coordinates (in meters)
    :param latitude:  list or np.array (shape: (n,))
    :param longitude: list or np.array (shape: (n,))
    :param geozone_num: geozone integer
    :param geozone_let: geozone letter
    :return: np.array of (x, y) coordinates (shape: (n, 2))
    """
    xy = np.array([
        utm.from_latlon(
            latitude=lat, 
            longitude=lon, 
            force_zone_number=geozone_num,
            force_zone_letter=geozone_let
        )[:2] for lat, lon in zip(latitude, longitude)
    ])
    return xy


def get_geozone(latitude, longitude):
    """
    :param latitude: 
    :param longitude: 
    :return: UTM geozone number and letter
    """
    geozone_num, geozone_let = utm.from_latlon(latitude, longitude)[2:]
    return geozone_num, geozone_let

In [3]:
def get_area_and_distance(
    tool_width : float,
    time : list,
    track_lat : list,
    track_lon : list,
    path_distance : list,
    field_processed : list,
    field_bounds : list
):
    """
    calculate path distance and processed field area
    :param tool_width: width of the tool (in meters)
    :param track_lat: list or np.array (shape: (n,))
    :param track_lon: list or np.array (shape: (n,))
    :param path_distance: list of pre-calculated path distances (shape: (m,))
    :param field_processed: list of pre-calculated processed area (shape: (m,))
    :param field_bounds: list of GPS subfields bounds
    :return: dictionary with calculated processed area and path distance
    """
    # get working UTM zone number and letter
    geozone_num, geozone_let = get_geozone(track_lat[0], track_lon[0])
    
    # define starting index of calculation
    start_from = len(field_processed)
    
    # convert GPS coordinates of the track
    track_xy = from_latlon(track_lat[start_from:], track_lon[start_from:], geozone_num, geozone_let)
    
    # calculate distances and areas
    distances = np.sqrt((track_xy**2).sum(axis=1))
    areas = distances * tool_width
    
    # append calculated distances and areas
    for i in range(len(areas)):
        path_distance.append(path_distance[-1] + distances[i])
        field_processed.append(field_processed[-1] + areas[i])
        
    # create dictionary with output
    area_calc = {
        'tool_width': tool_width,
        'time': time,
        'lat': track_lat,
        'long': track_lon,
        'path_distance': path_distance,
        'field_processed': field_processed
    }

    return area_calc

In [4]:
def update_area_calc(area_calc : dict, field_bounds_str : str):
    """
    update area_calc state 
    :param area_calc: dictionary of the field processing state
    :param field_bounds_str: string representation of the bounds
    :return: updated dictionary of the field processing state 
    
    """
    field_bounds = get_bounds_from_string(field_bounds_str)
    area_calc = get_area_and_distance(
        tool_width=area_calc['tool_width'],
        time=area_calc['time'],
        track_lat=area_calc['lat'],
        track_lon=area_calc['long'],
        path_distance=area_calc['path_distance'],
        field_processed=area_calc['field_processed'],
        field_bounds=field_bounds
    )
    return area_calc